In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 19506040
paper_name = 'burston_conibear_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/JCB_200811116_TS1.xlsx', sheet_name='TableS1', skiprows=3)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 2346 x 7


In [10]:
original_data.head()

,Rank,ORF,Gene,Normalized densitometry values,Unnamed: 4,Unnamed: 5,Score
0,NaN,NaN,NaN,MATa,MATα,Average,NaN
1,1.0,YCR009C,RVS161,0.10915,0.0378177,0.0734837,0.931168
2,2.0,YDR207C,UME6,0.0481679,0.0997867,0.0739773,0.930675
3,3.0,YOR035C,SHE4,0.141684,0.0750028,0.108343,0.896308
4,4.0,YLR337C,VRP1,0.140511,0.123529,0.13202,0.872632


In [11]:
original_data['orf'] = original_data['ORF'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

             Rank  ORF Gene Normalized densitometry values Unnamed: 4  \
index_input                                                             
0             NaN  NaN  NaN                          MATa        MATα   

            Unnamed: 5  Score  orf  
index_input                         
0              Average    NaN  NAN  


In [15]:
original_data = original_data.loc[t,:]

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['Normalized densitometry values','Unnamed: 4']].copy()

In [18]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(2345, 2)

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [123,536]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,123,536
data_type,value,value
orf,,
YAL004W,0.859942,1.006972
YAL005C,0.764880,0.919395
YAL007C,0.923195,0.978049
YAL010C,0.711928,NaN
YAL013W,0.132371,NaN


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 8


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,123,536
,data_type,value,value
gene_id,orf,,
1863,YAL004W,0.859942,1.006972
4,YAL005C,0.764880,0.919395
5,YAL007C,0.923195,0.978049
8,YAL010C,0.711928,NaN
11,YAL013W,0.132371,NaN


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,123,536,123,536
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
1863,YAL004W,0.859942,1.006972,1.409842,1.631859
4,YAL005C,0.764880,0.919395,1.253992,1.489935
5,YAL007C,0.923195,0.978049,1.513543,1.584987
8,YAL010C,0.711928,NaN,1.167178,NaN
11,YAL013W,0.132371,NaN,0.217017,NaN


# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [31]:
from IO.save_data_to_db3 import *

In [32]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 19506040...
Inserting the new data...


100%|██████████| 2/2 [00:08<00:00,  4.45s/it]

Updating the data_modified_on field...
